<a href="https://colab.research.google.com/github/romerik/tensorflow_practice/blob/main/C1_W4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

**Things to Note:**
1. When coding the `class myCallback`, Python 3 will run into an error
```python
TypeError: '>' not supported between instances of 'NoneType' and 'float'
```
when using the code
```python
if(logs.get('accuracy')>0.99):
```

For Python 3, use the following equivalent code line

```python
if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
```

2. You can run the notebook using TensorFlow 2.5.0

In [ ]:
#!pip install tensorflow==2.5.0

In [ ]:
# this is used in downloading data from the google drive
#!pip install gdown

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [ ]:
import tensorflow as tf
import os
import zipfile

!gdown --id 1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf 

zip_ref = zipfile.ZipFile("./happy-or-sad.zip", 'r')
zip_ref.extractall("./h-or-s")
zip_ref.close()

Downloading...
From: https://drive.google.com/uc?id=1NvV6VhmrfU8JDZNoEbwJxwx_6dhyN5bf
To: /content/happy-or-sad.zip
100% 2.67M/2.67M [00:00<00:00, 83.1MB/s]


In [ ]:
# GRADED FUNCTION: train_happy_sad_model
def train_happy_sad_model():
    # Please write your code only where you are indicated.
    # please do not remove # model fitting inline comments.

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') is not None and logs.get('acc')>0.999:
          print("Reached 99.9% accuracy so cancelling training!")
          self.model.stop_training = True
        # YOUR CODE END HERE
        
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',optimizer=RMSprop(learning_rate=0.001),metrics=['acc']
                 )
    
    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255.0)

    # Please use a target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory( 
        './h-or-s',
        target_size=(150,150),
        batch_size=16,
        class_mode='binary'
                                                       )
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit(train_generator,steps_per_epoch=5,epochs=15,verbose=1, callbacks=[callbacks]
                       )
    
    return history.history['acc'][-1]

In [ ]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
5/5 [==============================] - 5s 873ms/step - loss: 9.5679 - acc: 0.4500
Epoch 2/15
5/5 [==============================] - 4s 866ms/step - loss: 0.7104 - acc: 0.5500
Epoch 3/15
5/5 [==============================] - 4s 870ms/step - loss: 0.6238 - acc: 0.7000
Epoch 4/15
5/5 [==============================] - 4s 865ms/step - loss: 0.6352 - acc: 0.6625
Epoch 5/15
5/5 [==============================] - 4s 867ms/step - loss: 0.4140 - acc: 0.7625
Epoch 6/15
5/5 [==============================] - 4s 865ms/step - loss: 0.7221 - acc: 0.6500
Epoch 7/15
5/5 [==============================] - 4s 864ms/step - loss: 0.2808 - acc: 0.9125
Epoch 8/15
5/5 [==============================] - 4s 872ms/step - loss: 0.2205 - acc: 0.9125
Epoch 9/15
5/5 [==============================] - 4s 865ms/step - loss: 0.1663 - acc: 0.9500
Epoch 10/15
5/5 [==============================] - 4s 872ms/step - loss: 0.1133 - acc: 0.9625
Epoch 11/15
5/5 [============

1.0